So this colab aims to create a EfficientNetB4 model for 10 class food dataset and compare with B0

The data i get is Daniel Bourke's food data which is divided in to 3 datasets 1 percent, 10 percent and all.

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-08-31 10:31:45--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-31 10:31:46 (78.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
#importing dependencies
import tensorflow as tf
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

# Getting Data ready

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-08-31 10:34:40--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.128, 142.251.33.208, 142.250.188.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   138MB/s    in 1.2s    

2022-08-31 10:34:41 (138 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [5]:
unzip_data("10_food_classes_10_percent.zip")

In [6]:
walk_through_dir("10_food_classes_10_percent/")

There are 2 directories and 0 images in '10_food_classes_10_percent/'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/tes

In [7]:
# 75 images for training 250 images for testing

train_dir_10 = "10_food_classes_10_percent/train/"
test_dir_10 = "10_food_classes_10_percent/test/"

In [8]:
IMG_SIZE = (224, 224)
input_shape = (224, 224, 3)
train_data_10 = tf.keras.utils.image_dataset_from_directory(train_dir_10,
                                                            label_mode="categorical",
                                                            image_size=IMG_SIZE)

test_data = tf. keras.utils.image_dataset_from_directory(test_dir_10,
                                                         label_mode="categorical",
                                                         image_size=IMG_SIZE) # test_data will be always same so i am just attending it once 

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [9]:
train_data_10.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

# Building model

##### We got data ready now its time to build model i am going to build 4 different models
- model_1 is going to be our base model every tuning will be on this one.
- model_2 is going to be only fitted by augmented data.
- model_3 is going to be augmented data + tuning some layers on model_1 (which is not trainable on model_1)
- model_4 is going to be using all data + model_3 tunings

In [11]:
model = tf.keras.applications.EfficientNetB4(include_top=False)
model.trainable = False # freezing all the layers in model so that we can use them directly

inputs = tf.keras.layers.Input(shape=input_shape) # this is our input layer
x = model(inputs) # inputs passed to the model

# its time pool before outputting

x = tf.keras.layers.GlobalAveragePooling2D()(x) # tf.reduce_mean(inputs, axis=[1, 2])

outputs = tf.keras.layers.Dense(10, activation="softmax")(x) # after pooled get outputs to dense layer

71696384/71686520 [==============================] - 1s 0us/step


In [12]:
model_0 = tf.keras.Model(inputs, outputs)

model_0.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [13]:
model_0.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb4 (Functional)  (None, None, None, 1792)  17673823 
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 10)                17930     
                                                                 
Total params: 17,691,753
Trainable params: 17,930
Non-trainable params: 17,673,823
_________________________________________________________________
